In [1]:
import numpy as np
import pandas as pd

In [2]:
DATA  = 'incidents'
LABEL = 'hazard'
SHOTS = 2

# Load data:

In [3]:
results = pd.read_csv(f'../results/palm/palm_{LABEL}_{SHOTS:d}-shot.csv', index_col=0).fillna('')

In [4]:
prompt_cols = [col for col in results.columns if col.startswith('prompt_')]
results.head()

,cv_split,prompt_all,prompt_limited,prompt_conformal_20%,prompt_max-5,prompt_max-10,prompt_max-15,label,output_limited,output_conformal_20%,output_max-5,output_max-10
0,0,,Context start:\nWe are looking for food hazard...,Context start:\nWe are looking for food hazard...,Context start:\nWe are looking for food hazard...,Context start:\nWe are looking for food hazard...,Context start:\nWe are looking for food hazard...,mislabelled,misbranding,misbranding,,
1,0,,Context start:\nWe are looking for food hazard...,Context start:\nWe are looking for food hazard...,Context start:\nWe are looking for food hazard...,Context start:\nWe are looking for food hazard...,Context start:\nWe are looking for food hazard...,pistachio nut,,allergens,,
2,0,,Context start:\nWe are looking for food hazard...,Context start:\nWe are looking for food hazard...,Context start:\nWe are looking for food hazard...,Context start:\nWe are looking for food hazard...,Context start:\nWe are looking for food hazard...,illegal import,undeclared constituent,attempt to illegally import,,
3,0,,Context start:\nWe are looking for food hazard...,Context start:\nWe are looking for food hazard...,Context start:\nWe are looking for food hazard...,Context start:\nWe are looking for food hazard...,Context start:\nWe are looking for food hazard...,allergens,,allergens,,
4,0,,Context start:\nWe are looking for food hazard...,Context start:\nWe are looking for food hazard...,Context start:\nWe are looking for food hazard...,Context start:\nWe are looking for food hazard...,Context start:\nWe are looking for food hazard...,plastic fragment,,foreign bodies,,


# Sanity Checks:

In [5]:
for row in results[prompt_cols].values:
    last_text = None
    for text in row:
        if text != '':
            text = text.split('\nContext end:\n')[1]
            text = text.split('\n')[1]
            text = text[1:-5]
            
            if last_text is not None: assert text == last_text
            
            last_text = text

# Count unique classes per prompt:

In [6]:
class_count = pd.DataFrame(columns = ['cv_split'] + prompt_cols)
class_count['cv_split'] = np.arange(5)

for cv_split in class_count.cv_split:
    for col in prompt_cols:
        counts = []
        for prompt in results[results.cv_split==cv_split][col].values:
            prompt = prompt.split('\nContext end:')[0]
            prompt = prompt.split('\n')[2:]
            labels = set([sample.split(' -> ')[1] for sample in prompt])
            
            counts.append(len(labels))

        class_count.loc[class_count.cv_split == cv_split, col] = np.mean(counts)

class_count.set_index('cv_split', inplace=True)
class_count

,prompt_all,prompt_limited,prompt_conformal_20%,prompt_max-5,prompt_max-10,prompt_max-15
cv_split,,,,,,
0,0.0,15.455381,7.783465,5.0,10.0,15.0
1,0.0,19.586614,9.919291,5.0,10.0,15.0
2,0.0,20.891732,10.557087,5.0,10.0,15.0
3,0.0,19.103743,9.64281,5.0,10.0,15.0
4,0.0,17.425476,8.801707,5.0,10.0,15.0


In [7]:
class_count.mean()

prompt_all                    0.0
prompt_limited          18.492589
prompt_conformal_20%     9.340872
prompt_max-5                  5.0
prompt_max-10                10.0
prompt_max-15                15.0
dtype: object

# Count Characters per prompt:

In [8]:
char_count = pd.DataFrame(columns = ['cv_split'] + prompt_cols)
char_count['cv_split'] = np.arange(5)

for cv_split in char_count.cv_split:
    for col in prompt_cols:
        counts = []
        for prompt in results[results.cv_split==cv_split][col].values:
            counts.append(len(prompt))

        char_count.loc[char_count.cv_split == cv_split, col] = np.mean(counts)

char_count.set_index('cv_split', inplace=True)
char_count

,prompt_all,prompt_limited,prompt_conformal_20%,prompt_max-5,prompt_max-10,prompt_max-15
cv_split,,,,,,
0,0.0,1936.572178,2048.619423,1434.16601,2532.165354,3584.071522
1,0.0,2374.441601,2473.2979,1420.72769,2498.456037,3535.564304
2,0.0,2496.899606,2622.363517,1436.853675,2516.720472,3549.921916
3,0.0,2306.018385,2448.878529,1425.849639,2524.424819,3588.776756
4,0.0,2126.35916,2250.261983,1429.349311,2514.719632,3553.996717


In [9]:
char_count.mean()

prompt_all                      0.0
prompt_limited          2248.058186
prompt_conformal_20%     2368.68427
prompt_max-5            1429.389265
prompt_max-10           2517.297263
prompt_max-15           3562.466243
dtype: object